In [ ]:
import os, sys, subprocess

REPO = "https://github.com/jongmoonha/Signal_Processing_Practice.git"
DIR  = "Signal_Processing_Practice"

try:
    import google.colab  # Colab 전용 모듈
    if not os.path.isdir(DIR):
        subprocess.run(["git","clone",REPO], check=True)
    print('We are in Google Colab environment.')
    os.chdir('/content/'+DIR)
    print('Current working directory:', os.getcwd())

except ImportError:
    print('We are in a local environment, not Google Colab.')
    pass

# Resampling, Order analysis, TSA Practice

In [ ]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np

from scipy import signal, stats
from scipy.signal import hilbert
from scipy.signal import lfilter
from scipy.interpolate import interp1d

import utils

plt.rc('font', size=20)

## 신호 생성

In [ ]:
fs = 1000;
T = 20;
t = np.arange(1/fs,T+1/fs,1/fs);
n_gear = 3

w_rot = 1+t

In [ ]:
phase_deg = np.cumsum(w_rot)*360/fs
phase_rad = 2*np.pi*phase_deg/360
v = np.sin(n_gear*phase_rad);

In [ ]:
plt.figure(figsize=(8,6))
plt.subplot(211)
plt.plot(t,w_rot);
plt.xlabel('Time (s)');plt.ylabel('Speed (Hz)')
plt.subplot(212)
plt.plot(t,phase_deg)
plt.xlabel('Time (s)');plt.ylabel('deg')
plt.subplots_adjust(hspace=0.5)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(t,v,'.-');
plt.xlabel('Time (s)');plt.ylabel('v')

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(t,v,'.-');
plt.xlabel('Time (s)');plt.ylabel('v')
plt.xlim([0, 5])

## 주파수 분석

In [ ]:
[f, A] = utils.fft_simple(v,fs);
plt.figure(figsize=(8,6))
plt.plot(f,A)
plt.xlabel('Freq');plt.ylabel('Y')

## Resampling Frequency 결정

### 데이터에서 계산해도 되고, 그냥 정해줘도 됨

In [ ]:
Nrot = phase_deg[-1]/360
Nsamples = len(v)
fs_re = np.floor(Nsamples/Nrot)

print('# of Rotations are {:.2f}'.format(Nrot))
print('# of Samples are {:.0f}'.format(Nsamples))
print('# of Samples per rotation are {:.2f}'.format(fs_re))

In [ ]:
fs_re = 100

## Resampling 코드 짜보기

In [ ]:
starting = phase_deg[0]
ending = phase_deg[-1]

In [ ]:
degree_re_delta = 360 / fs_re

In [ ]:
degree_re = np.arange(starting + degree_re_delta,  ending,  degree_re_delta)

degree_re[0:4]

In [ ]:
degree_re[-7:-1]

In [ ]:
fx_t_re = interp1d(phase_deg, t);
t_re=fx_t_re(degree_re)

fx_v_re = interp1d(t, v);
v_re=fx_v_re(t_re)

Order analysis


In [ ]:
[order, order_A] = utils.fft_simple(v_re,fs_re);

plt.figure(figsize=(8,6))
plt.subplot(121);plt.stem(order,order_A);plt.xlabel('Order');plt.ylabel('Y')
plt.subplot(122);plt.stem(order,order_A);plt.xlim([2,4]);plt.xlabel('Order');plt.ylabel('Y')

In [ ]:
plt.figure(figsize=(8,6))
plt.subplot(121);plt.plot(order,order_A);plt.xlabel('Order');plt.ylabel('Y')
plt.subplot(122);plt.plot(order,order_A);plt.xlim([2,4]);plt.xlabel('Order');plt.ylabel('Y')

## Resampling - Trig=0

신호 전체 길이가 정확히 회전수의 정배수가 아니다 => 주파수 해상도가 소수점

In [ ]:
trig = 0
[t_re, v_re, degree_re, fs_re]=utils.resampling(t,v,phase_deg,fs_re,trig)
N_re=len(v_re)
print(N_re)

## Order analysis

In [ ]:
order, order_A = utils.fft_simple(v_re,fs_re);
plt.figure(figsize=(8,6))
plt.subplot(121);plt.plot(order,order_A);plt.xlabel('Order');plt.ylabel('Y')
plt.subplot(122);plt.plot(order,order_A,'-o');plt.xlim([2.95,3.05]);plt.xlabel('Order');plt.ylabel('Y')

## 해결방법 1: Zero-padding으로 회전수 정수배를 채워주자

In [ ]:
v_re_pad = np.append(v_re, (np.zeros(fs_re-np.mod(N_re,fs_re))))
len(v_re_pad)

In [ ]:
order, order_A = utils.fft_simple(v_re_pad,fs_re);
plt.figure(figsize=(8,6))
plt.subplot(121);plt.plot(order,order_A);plt.xlabel('Order');plt.ylabel('Y')
plt.subplot(122);plt.plot(order,order_A,'-o');plt.xlim([2.95,3.05]);plt.xlabel('Order');plt.ylabel('Y')

## 해결방법 2: 회전수 정수배가 되도록 신호 마지막을 버려주자
## Resampling - Trig=1 -> 정확히 n 바퀴에 해당하는 진동 신호만 추출

In [ ]:
trig = 1
t_re, v_re, degree_re, fs_re=utils.resampling(t,v,phase_deg,fs_re,trig)
N_re=len(v_re)
print(N_re)

Order analysis


In [ ]:
order, order_A = utils.fft_simple(v_re,fs_re)
plt.figure(figsize=(8,6))
plt.subplot(121);plt.plot(order,order_A);plt.xlabel('Order');plt.ylabel('Y')
plt.subplot(122);plt.plot(order,order_A,'-o');plt.xlim([2.95,3.05]);plt.xlabel('Order');plt.ylabel('Y')

# TSA 및 고장진단

## 신호생성
### 만약 신호 안에 주기적으로 (기어 1바퀴당 1번의) 이상치가 있었다면

In [ ]:
v_re_f = np.copy(v_re_pad)
v_re_n = np.copy(v_re_f)
idx_tmp=np.arange(40,len(v_re_f),fs_re)
v_re_f[idx_tmp] = v_re_f[idx_tmp] + 0.5

plt.figure(figsize=(8,6))
plt.plot(v_re_f)
plt.figure(figsize=(8,6))
plt.plot(v_re_f)
plt.plot(v_re_n,'k')
plt.xlim([0, 1000])

## 그런데 이 이상치들이 노이즈에 가려졌다면?

In [ ]:
# Fix the random seed for reproducibility
np.random.seed(1000)
v_re_f = v_re_f+np.random.randn(np.size(v_re_f))/2;
v_re_n = v_re_n+np.random.randn(np.size(v_re_n))/2;

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_re_f)
plt.figure(figsize=(8,6))
plt.plot(v_re_f)
plt.xlim([0, 1000])

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_re_f,'r'); plt.plot(v_re_n,'k')
plt.xlim([0,1000])
plt.show()

## 신호를 잘라서 평균화

In [ ]:
n_rot_TSA = 5
v_reshape_f = v_re_f.reshape(n_rot_TSA*fs_re, -1,  order="F")
v_TSA_f = np.mean(v_reshape_f,1)
np.shape(v_reshape_f)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_reshape_f, alpha=0.1)
plt.plot(v_TSA_f,'r--',linewidth=2.5)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_TSA_f)

## 여기서 다시 Feature를 뽑아보자
### 일단 비교를 위해 정상 데이터도 만들어 주고

In [ ]:
v_reshape_n = v_re_n.reshape(n_rot_TSA*fs_re, -1,  order="F")
v_TSA_n = np.mean(v_reshape_n,1)

### Plot Data

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_TSA_f,'r')
plt.plot(v_TSA_n,'b')
plt.xlabel('Samples');plt.ylabel('v')
plt.show()

### 주파수 분석해보기


In [ ]:
f_n, A_n = utils.fft_simple(v_TSA_n,fs_re)
f_f, A_f = utils.fft_simple(v_TSA_f,fs_re)
plt.figure(figsize=(8,6))
plt.plot(f_f,A_f,'r')
plt.plot(f_n, A_n, 'b')
plt.xlabel('Order');plt.ylabel('|Y|')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(f_f,A_f,'r')
plt.plot(f_n, A_n, 'b');plt.ylim([0,0.05])
plt.xlabel('Order');plt.ylabel('|Y|')

### 필터링 후 분석 해보기

Filter Design and Filtering


In [ ]:
ftype='high'
Wn=np.array([10])
print(Wn)
v_high_n = utils.filtering(v_TSA_n, fs_re, Wn, ftype)
v_high_f = utils.filtering(v_TSA_f, fs_re, Wn, ftype)

plt.figure(figsize=(8,6))
plt.plot(v_high_f,'r')
plt.plot(v_high_n,'b')
plt.xlabel('Samples');plt.ylabel('v')
plt.show()

### 필터링 신호의 Residual 보기

In [ ]:
v_res = abs(hilbert(v_high_f)) - abs(hilbert(v_high_n))
v_res = abs(hilbert(v_res))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_res,'r')
plt.xlabel('Samples');plt.ylabel('residual')